# Human coding

In [138]:
# get each coding

import glob
import pandas as pd
import numpy as np
from sklearn.metrics import cohen_kappa_score, accuracy_score

limit = 350

df_gpt = pd.read_excel('../data/GPT/sample_for_check_GPT3.xlsx').fillna('NA')[:limit]
df_elin = pd.read_excel('../data/GPT/human_annotation/ELINsample_for_check_human.xlsx').fillna('NA')[:limit]
df_lena = pd.read_excel('../data/GPT/human_annotation/Lenasample_for_check_human.xlsx').fillna('NA')[:limit]
df_oscar = pd.read_excel('../data/GPT/human_annotation/Oscarsample_for_check_human.xlsx').fillna('NA')[:limit]
masks = dict()

for dim in 'dim1 dim2 dim3'.split():
    names = ['elin', 'lena', 'oscar']

    print(dim)
    data = {'elin': df_elin[dim], 'lena': df_lena[dim], 'oscar': df_oscar[dim]}
    out = pd.DataFrame(data=data)
    print(out)
    
    if dim in {'dim2', 'dim3'}:
        masks[dim] = (out.elin != 'NA') & (out.lena!="NA") & (out.oscar!="NA") 
        out2 = out[masks[dim]]
        print(out2)
    else:
        out2 = out
    
    out2.to_excel('../data/GPT/human_annotation/{}.xlsx'.format(dim), index=None)
    out2.to_csv('../data/GPT/human_annotation/{}.csv'.format(dim), index=None, header=None)

dim1
                 elin               lena              oscar
0             PASSIVE            PASSIVE            PASSIVE
1             PASSIVE                 NA                 NA
2    ACTIVE_POS_OTHER   ACTIVE_POS_OTHER            PASSIVE
3    ACTIVE_POS_OTHER   ACTIVE_POS_OTHER   ACTIVE_POS_OTHER
4             PASSIVE            PASSIVE            PASSIVE
..                ...                ...                ...
345        ACTIVE_NEG         ACTIVE_NEG         ACTIVE_NEG
346                NA                 NA                 NA
347                NA            PASSIVE            PASSIVE
348  ACTIVE_POS_OTHER  ACTIVE_POS_CARING  ACTIVE_POS_CARING
349                NA   ACTIVE_POS_OTHER                 NA

[350 rows x 3 columns]
dim2
         elin      lena     oscar
0    EXPLICIT  EXPLICIT  EXPLICIT
1    EXPLICIT        NA        NA
2    IMPLICIT  EXPLICIT  EXPLICIT
3    EXPLICIT  EXPLICIT  EXPLICIT
4    EXPLICIT  EXPLICIT  EXPLICIT
..        ...       ...       ...
345  EXP

/Users/dirkhovy/anaconda3/lib/python3.8/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():
/Users/dirkhovy/anaconda3/lib/python3.8/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():
/Users/dirkhovy/anaconda3/lib/python3.8/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


In [130]:
! wc -l ../data/GPT/human_annotation/${i} ../data/GPT/human_annotation/*csv

wc: ../data/GPT/human_annotation/: read: Is a directory
     350 ../data/GPT/human_annotation/dim1.csv
     225 ../data/GPT/human_annotation/dim2.csv
     225 ../data/GPT/human_annotation/dim3.csv
     800 total


In [132]:
! /Users/dirkhovy/Dropbox/tools/MACE/MACE --prefix ../data/GPT/human_annotation/dim1 ../data/GPT/human_annotation/dim1.csv
! /Users/dirkhovy/Dropbox/tools/MACE/MACE --prefix ../data/GPT/human_annotation/dim2 ../data/GPT/human_annotation/dim2.csv
! /Users/dirkhovy/Dropbox/tools/MACE/MACE --prefix ../data/GPT/human_annotation/dim3 ../data/GPT/human_annotation/dim3.csv

Reading CSV file '../data/GPT/human_annotation/dim1.csv'
....................100
....................200
....................300
.........
stats:
	350 instances,
	6 labels [PASSIVE, NA, ACTIVE_POS_OTHER, ACTIVE_NEG, ACTIVE_POS_CHALLENGING, ACTIVE_POS_CARING],
	3 annotators

Running Variational Bayes EM training with the following settings:
	50 iterations
	10 restarts
	smoothing = 0.0016666666666666668
	alpha = 0.5
	beta = 0.5

Restart 1
initial log marginal likelihood = -1555.6882106001829
final log marginal likelihood = -1351.0009273885416

Restart 2
initial log marginal likelihood = -1553.1490471943941
final log marginal likelihood = -1351.0009277253812

Restart 3
initial log marginal likelihood = -1572.0079877178498
final log marginal likelihood = -1351.0009276184364

Restart 4
initial log marginal likelihood = -1542.5678174738118
final log marginal likelihood = -1351.0009269094794

Restart 5
initial log marginal likelihood = -1522.559650468198
final log marginal likelihood = -1351.

In [182]:
dim1_mode = '' #binary/reduced

for dim in 'dim1 dim2 dim3'.split():
    print(dim)
    with open('../data/GPT/human_annotation/{}.prediction'.format(dim)) as dimagg:
        dim_gpt_agg = [x.strip() for x in dimagg.readlines()]
    # change for reduced and binary dim1
    if dim == 'dim1':
        if dim1_mode == 'binary':
            dim_gpt_agg = [x[:3] for x in dim_gpt_agg]
        elif dim1_mode == 'reduced':
            dim_gpt_agg = [x[:10] for x in dim_gpt_agg]
        
    out = pd.read_excel('../data/GPT/human_annotation/{}.xlsx'.format(dim)).fillna('NA')
    gpt_label = df_gpt[dim].values
    # change for reduced and binary dim1
    if dim == 'dim1':
        if dim1_mode == 'binary':
            gpt_label = [x[:3] for x in gpt_label]
        elif dim1_mode == 'reduced':
            gpt_label = [x[:10] for x in gpt_label]
    # account for excluded items
    else:    
        gpt_label = df_gpt[dim][masks[dim]].values
  

    print(len(gpt_label))
    conf_kappa = pd.DataFrame(columns=names+['aggregate', 'GPT'], index=names+['aggregate', 'GPT']).fillna(1.0)
    conf_raw = pd.DataFrame(columns=names+['aggregate', 'GPT'], index=names+['aggregate', 'GPT']).fillna(1.0)

    for i, name1 in enumerate(names):
        label1 = out[name1].values
        if dim == 'dim1':
            if dim1_mode == 'binary':
                label1 = [x[:3] for x in label1]
            elif dim1_mode == 'reduced':
                label1 = [x[:10] for x in label1]
        
        ck = cohen_kappa_score(label1, gpt_label)
        agg = accuracy_score(label1, gpt_label)
        
        for j, name2 in enumerate(names[i+1:]):
            label2 = out[name2].values
            if dim == 'dim1':
                if dim1_mode == 'binary':
                    label2 = [x[:3] for x in label2]
                elif dim1_mode == 'reduced':
                    label2 = [x[:10] for x in label2]
                    
            if name1 != name2:
                ck1 = cohen_kappa_score(label1, label2)
                agg1 = accuracy_score(label1, label2)
                conf_kappa[name1][name2] = ck1
                conf_raw[name1][name2] = agg1
                conf_kappa[name2][name1] = ck1
                conf_raw[name2][name1] = agg1
                
        conf_kappa['GPT'][name1] = ck
        conf_raw['GPT'][name1] = agg
        conf_kappa[name1]['GPT'] = ck
        conf_raw[name1]['GPT'] = agg
        ck_label_agg = cohen_kappa_score(dim_gpt_agg, label1)
        agg_label_agg = accuracy_score(dim_gpt_agg, label1)
#         print('{}:aggregate, kappa={:.2f}, agreement={:.2f}'.format(name1, ck_label_agg, agg_label_agg))
        conf_kappa[name1]['aggregate'] = ck_label_agg
        conf_raw[name1]['aggregate'] = agg_label_agg
        conf_kappa['aggregate'][name1] = ck_label_agg
        conf_raw['aggregate'][name1] = agg_label_agg

    ck_agg = cohen_kappa_score(dim_gpt_agg, gpt_label)
    agg_agg = accuracy_score(dim_gpt_agg, gpt_label)
#     print('aggregate:GPT, kappa={:.2f}, agreement={:.2f}'.format(ck_agg, agg_agg))
    conf_kappa['GPT']['aggregate'] = ck_agg
    conf_raw['GPT']['aggregate'] = agg_agg
    conf_kappa['aggregate']['GPT'] = ck_agg
    conf_raw['aggregate']['GPT'] = agg_agg
    
    conf_kappa['mean_human'] = conf_kappa[names].mean(axis=1)
    conf_raw['mean_human'] = conf_raw[names].mean(axis=1)
    for name in names:
        conf_kappa.mean_human[name] = (conf_kappa[names].loc[name].sum() - 1.0) / 2
        conf_raw.mean_human[name] = (conf_raw[names].loc[name].sum() - 1.0) / 2
        
    print('RAW:')
    print(conf_raw)
    print('KAPPA:')
    print(conf_kappa)
    print()

dim1
350
RAW:
               elin      lena     oscar  aggregate       GPT  mean_human
elin       1.000000  0.608571  0.617143   0.748571  0.494286    0.612857
lena       0.608571  1.000000  0.668571   0.817143  0.545714    0.638571
oscar      0.617143  0.668571  1.000000   0.840000  0.528571    0.642857
aggregate  0.748571  0.817143  0.840000   1.000000  0.548571    0.801905
GPT        0.494286  0.545714  0.528571   0.548571  1.000000    0.522857
KAPPA:
               elin      lena     oscar  aggregate       GPT  mean_human
elin       1.000000  0.486754  0.499290   0.672779  0.354546    0.493022
lena       0.486754  1.000000  0.533719   0.750879  0.388609    0.510237
oscar      0.499290  0.533719  1.000000   0.781927  0.364316    0.516505
aggregate  0.672779  0.750879  0.781927   1.000000  0.406302    0.735195
GPT        0.354546  0.388609  0.364316   0.406302  1.000000    0.369157

dim2
225
RAW:
               elin      lena     oscar  aggregate       GPT  mean_human
elin       1.00

In [181]:
! cat ../data/GPT/human_annotation/*competence

0.6979719345102646	0.701234171762788	0.7207665075440628
0.35914902160911977	0.03159441821690089	0.42524136551048913
0.9920064209387123	0.27533097091128145	0.5968191612149533


In [178]:
(conf_kappa[names].loc[name].sum() - 1.0)/2

0.49199199199199195

In [118]:
from collections import Counter
outcomes = []
for d1 in range(1,7):
    for d2 in range(1,7):
        for d3 in range(1,7):
            outcomes.extend([d1+d2, d1+d3, d2+d3])
print(Counter(outcomes))

Counter({7: 108, 6: 90, 8: 90, 5: 72, 9: 72, 4: 54, 10: 54, 3: 36, 11: 36, 2: 18, 12: 18})


In [157]:
conf['elin']['lena']

1.0